## Visualization   
read the data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import geopandas as gpd
from matplotlib.animation import FuncAnimation
import folium
from folium.plugins import HeatMap
import matplotlib.ticker as ticker
import seaborn as sns
import folium as fol
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium.plugins import Fullscreen
#!pip install geopandas
#!pip install geopy
from geopy.distance import geodesic
from shapely.geometry import Point
import os

from collections import defaultdict, Counter
from typing import Union
import warnings
df = pd.read_csv('cleaned_meteorite_landing_from_1950.csv')
df.head()

# Box-plot of distribution of year

In [ ]:
def style(box, figure=None):
    #set figurecolor
    if figure:
        figure.set_facecolor('#2b3e50')
    
    # set color
    box.set_facecolor('#566573')
    box.tick_params(axis='both', colors='white')
    box.xaxis.label.set_color('white')
    box.yaxis.label.set_color('white')
    box.title.set_color('white')
    
    # Set size
    box.tick_params(axis='y', labelsize=14)
    box.tick_params(axis='x', labelsize=12)
    box.set_xlabel(box.get_xlabel(), fontsize=14)
    box.set_ylabel(box.get_ylabel(), fontsize=14)
figure = plt.figure(figsize=(10, 5))
box = sns.boxplot(data=df, x='fall', y='year')
style(box, figure)
plt.ylim(1950, 2020)
plt.title('Distribution of Year Fell vs Found', size=18)

# histplot of fall/found data on year

In [ ]:
# set cut off years
cut_off = 1950
end = 2020

# filter data
found_data = df[(df['fall'] == 'Found') & (df['year'].notnull()) & (df['year'] >= cut_off)]
fell_data = df[(df['fall'] == 'Fell') & (df['year'].notnull()) & (df['year'] >= cut_off)]

#adjust bins
bins = [cut_off + i*10 for i in range((end - cut_off)//10 + 1)]
bins_year = [cut_off + i for i in range(end - cut_off + 1)]


figure, _ = plt.subplots(2, 2, figsize=(15, 8))

plt.subplot(2, 2, 1)
hist = sns.histplot(data=found_data, x='year', bins=bins)
style(hist, figure)
plt.xlim(cut_off, end)
plt.title('Found Meteorites(by decade)', size=18, color='white')

plt.subplot(2, 2, 2)
hist = sns.histplot(data=found_data, x='year', bins=bins_year)
style(hist, figure)
plt.xlim(cut_off, end)
plt.title('Found Meteorites(by year)', size=18, color='white')

plt.subplot(2, 2, 3)
hist = sns.histplot(data=fell_data, x='year', bins=bins, color='orange')
style(hist)
plt.xlim(cut_off, end)
plt.title('Fell Meteorites(by decade)', size=18, color='white')

plt.subplot(2, 2, 4)
hist = sns.histplot(data=fell_data, x='year', bins=bins_year, color='orange')
style(hist)
plt.xlim(cut_off, end)
plt.title('Fell Meteorites(by year)', size=18, color='white')

plt.suptitle(f'Distribution of Meteorites({cut_off}-{end})', size=24, color='white')
plt.tight_layout()

# plot of distribution location

In [ ]:
pd.concat([df['latitude'].describe().to_frame(), df['longitude'].describe().to_frame()], axis=1)

In [ ]:
found_data = df[df['fall'] == 'Found']
fell_data = df[df['fall'] == 'Fell']

figure = plt.figure(figsize=(12.8, 4.8))

plt.subplot(1, 2, 1)
scat = sns.scatterplot(data=df, x='longitude', y='latitude', edgecolor='black')
style(scat, figure)
plt.title('All Meteorite locations', size=18, color='white')

plt.subplot(1, 2, 2)
scat = sns.scatterplot(data=found_data, x='longitude', y='latitude', edgecolor='black', label='Found')
style(scat, figure)
plt.title('Meteorite Locations by Type', size=18, color='white')

scat = sns.scatterplot(data=fell_data, x='longitude', y='latitude', alpha = 0.5, edgecolor='black', label='Fell')
style(scat)
plt.legend(bbox_to_anchor=(1, 1))
plt.tight_layout()
print(figure.get_size_inches())

# make a animation of meteorite landings of their location by different years
I didnt find a good world backrgound map for it^^, so the backrgound is blank

In [ ]:
#make a animation

#Load world map
#world = gpd.read_file('C:/Users/Lenovo/Downloads/ne_110m_admin_0_boundary_lines_land/ne_110m_admin_0_map_units.shp')
#Filter data for valid years
df['year'] = df['year'].astype(int)
df = df[(df['year'] >= 1950) & (df['year'] <= 2020)]

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))
# Plot the world map
#world.boundary.plot(ax=ax)
# Function to update the plot for each frame
def update(year):
    ax.clear()
    #world.boundary.plot(ax=ax)
    yearly_data = df[df['year'] == year]
    ax.scatter(yearly_data['longitude'], yearly_data['latitude'], s=10, c='red', alpha=0.6)
    ax.set_title(f'Meteorite Landings in {year}', fontsize=15)
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)

# Create the animation
years = sorted(df['year'].unique())
ani = FuncAnimation(fig, update, frames=years, repeat=False)

# Save the animation as a gif
ani.save('meteorite_landings.gif', writer='imagemagick')

plt.show()

after we have the population data, we can combine them together

In [ ]:
# Load world map with population density data
#TODO: Load world map data
#world = gpd.read_file('')
#TODO: Load population density data
population_density = pd.read_csv('path_to_population_density_data.csv')  # Ensure this CSV contains 'country', 'year', 'population_density'

# Merge population density data with world map
world = world.merge(population_density, how='left', left_on='name', right_on='country')

# Filter data for valid years
df['year'] = df['year'].astype(int)


# Create a figure and axis
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# Function to update the plot for each frame
def update(year):
    ax.clear()
    yearly_data = df[df['year'] == year]
    world_year = world[world['year'] == year]
    
    # Plot the world map with population density
    world_year.plot(column='population_density', ax=ax, legend=True, cmap='OrRd', legend_kwds={'label': "Population Density by Country"})
    
    # Plot meteorite landings
    ax.scatter(yearly_data['longitude'], yearly_data['latitude'], s=10, c='red', alpha=0.6)
    ax.set_title(f'Meteorite Landings in {year}', fontsize=15)
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)

# Create the animation
years = sorted(df['year'].unique())
ani = FuncAnimation(fig, update, frames=years, repeat=False)

# Save the animation as a gif
ani.save('meteorite_landings_with_population_density.gif', writer='imagemagick')

plt.show()

In [ ]:
#!pip install statsmodels --user
#from statsmodels.nonparametric.smoothers_lowess import lowess
df.columns

# Heatmap on location

In [ ]:
import folium as fol
def choose_color(fall_type: str) -> str:
    if fall_type == 'Found':
        return 'blue'
    elif fall_type == 'Fell':
        return 'orange'
    else:
        return 'red'

# convert mass to Kg depending on size
def convert_mass(mass: float) -> str:
    if pd.isnull(mass):
        return mass
    if mass < 1000:
        return f'{mass}g'
    else:
        mass = mass / 1000
        return f'{round(mass, 2)}Kg'


# convert to year to int when possible
def convert_int(year: float):
    try:
        return int(year)
    except:
        return year
    
# A function to create the tool tip string for the marker.
def tool_tip_string(row: pd.Series) -> str:
    tool_tip = f'''name: {row['name']}, id: {row['id']}, type: {row['type']}, class: {row['classification']}<br>
                   mass: {convert_mass(row['mass'])}, fall: {row['fall']}, year: {convert_int(row['year'])}, 
                   latitude: {row['latitude']}, longitude: {row['longitude']}'''
    return tool_tip
        

# clean data set to remove nan values:
c_df = df[df['latitude'].notnull() & df['longitude'].notnull() & df['fall'].notnull()]


earth = fol.Map(world_copy_jump=True)
found = fol.FeatureGroup(name='Found Meteorites').add_to(earth)
marker_cluster_found = MarkerCluster(disableClusteringAtZoom=12, maxClusterRadius=20).add_to(found)
fell = fol.FeatureGroup(name='Observed Meteorites').add_to(earth)
marker_cluster_fell = MarkerCluster(disableClusteringAtZoom=10, maxClusterRadius=20).add_to(fell)



for i, row in c_df.iterrows():
    if row['fall'] == 'Found':
        fol.Marker([row['latitude'], row['longitude']], 
                   icon=fol.Icon(color=choose_color(row['fall']), icon='eye-close'),
                   tooltip = tool_tip_string(row)).add_to(marker_cluster_found)
    else:
        fol.Marker([row['latitude'], row['longitude']], 
                   icon=fol.Icon(color=choose_color(row['fall']), icon='eye-open'),
                   tooltip = tool_tip_string(row)).add_to(marker_cluster_fell)

fullscreen = Fullscreen()
fullscreen.add_to(earth)
fol.LayerControl().add_to(earth)
earth.save('meteorite_location_map.html')
earth

In [15]:
heat_data_found = []
heat_data_fell = []
all_data = []
for i, row in c_df.iterrows():
    if row['fall'] == 'Found':
        heat_data_found.append([row['latitude'], row['longitude']])
        all_data.append([row['latitude'], row['longitude']])
    else:
        heat_data_fell.append([row['latitude'], row['longitude']])
        all_data.append([row['latitude'], row['longitude']])

In [ ]:
len(heat_data_found),len(heat_data_fell)

In [ ]:
#draw heat map
earth = fol.Map()
gradient = gradient = {0.2: 'blue', 0.3: 'cyan', 0.4: 'lime', 0.5: 'yellow', 0.6: 'orange', 0.9: 'red'}
HeatMap(heat_data_found, name='Found Meteorites', radius=10,gradient=gradient).add_to(earth)
HeatMap(heat_data_fell, name='Observed Meteorites', radius=10,gradient=gradient).add_to(earth)
HeatMap(all_data, name='All Meteorites', radius=10,gradient=gradient).add_to(earth)
fullscreen = Fullscreen()
fullscreen.add_to(earth)
fol.LayerControl().add_to(earth)
earth.save('meteorite_heat_map.html')
earth


# nearest meteorites to tuebingen

In [18]:
# set coordinates for your location.
import geopy
coor = [48.521637, 9.057645]
#coor = [50, 0]

def calc_distance(row: pd.Series, location: list[int]) -> int:
    return geodesic([row['latitude'], row['longitude']], location).kilometers

geo_df = gpd.GeoDataFrame(c_df, geometry=gpd.points_from_xy(c_df['latitude'], c_df['longitude']))
geo_df['distance'] = geo_df.apply(calc_distance, location=coor, axis=1)
geo_df_top_ten = geo_df.sort_values(by='distance', ascending=True).head(10)

top_ten = fol.Map(location=coor, world_copy_jump=True)
fol.Marker(location=coor, icon=fol.Icon(color='green', icon='user'), 
           tooltip = 'Your Location').add_to(top_ten)

for i, row in geo_df_top_ten.iterrows():
    if row['fall'] == 'Found':
        fol.Marker([row['latitude'], row['longitude']], 
                   icon=fol.Icon(color='blue', icon='eye-close'),
                   tooltip = f"Distance: {round(row['distance'], 2)}Km<br>" + tool_tip_string(row)).add_to(top_ten)
        fol.PolyLine([coor, [row['latitude'], row['longitude']]], color='red').add_to(top_ten)
    else:
        fol.Marker([row['latitude'], row['longitude']], 
                   icon=fol.Icon(color='orange', icon='eye-open'),
                   tooltip = f"Distance: {round(row['distance'], 2)}Km<br>" + tool_tip_string(row)).add_to(top_ten)
        fol.PolyLine([coor, [row['latitude'], row['longitude']]], color='red').add_to(top_ten)

print(coor)
top_ten.save('top ten closest meteorites (tubingen).html')
top_ten

[48.521637, 9.057645]
